<a href="https://colab.research.google.com/github/Multik838/-Data-analysis-algorithms/blob/main/%D0%9F%D0%BE%D0%B4%D0%B3%D0%BE%D1%82%D0%BE%D0%B2%D0%BA%D0%B0_%D0%BA_%D1%83%D1%80%D0%BE%D0%BA%D1%83_4_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# Просмотрен весь ноутбук
# Загружены Train и Test
# Добавлен импорт import pyspark.sql.functions as F и переписана запись my_awesome_col = F.struct(F.col("id")).alias("foo")
# #df.dropDuplicates(subset=["battery_power", "dual_sim"]).show()

#df.show() # Нам показывает, что в тек. датафрейме остался один столбец ID.
# +---+
# | id|
# +---+
# |  0|
# |  1|
# |  2|
# |  3|
# |  4|
# |  5|
# |  6|
# |  7|
# |  8|
# |  9|
# +---+

#Поэтому перезапрашиваем df = spark.read.csv('train.csv', header=True, inferSchema=True), Либо JOIN 
#Drop Duplicates отработает

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 62.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=971e9f1e53c87e62b378864679fb0e91776158d61a3ff527e707a87869561ed0
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession


spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', f'{int(6000/1.1)}mb')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [3]:
spark

## Ядро Spark `RDD`:
  - Ядром Spark является абстракция данных под названием Resilient Distributed Datasets, которые являются отказоустойчивыми распределенными наборами данных
  - RDD являются статически типизированными, распределенными колекциями с отложенным вычислением
  - Наборы RDD реализованы в Spark как неизменяемые, так что преобразование объекта RDD возвращает новый объект, а не уже существующий.
  - Имеют предопределенные высокоуровневые преобразования (aka map, join, reduce) 
  - Функции записи данных между распределенной системой хранения и JVM Spark
  
## Ленивые вычисления:
  - Spark вычисляет RDD отложенным образом, рассчитывая результат преобразований набора только в момент, когда становятся необходимы итоговые данные RDD
  - Spark умеет хранить RDD в оперативной памяти Worker-ов во время всего времени жизни приложения Spark ради ускорения доступа при повторяемых вычислениях
  - **`Отложенные вычисления`** + **`хранения данных в оперативной памяти`** + **`неизменяемость объектов`** == **`эффективность`** + **`отказоустойчивость`** + **`масштабируемость`**

# DataFrame

## Сравнение RDD API и DataFrame API 

### Типы данных
**`RDD`**: низкоуревная распределенная коллекция данных любого типа  
**`DF`**: таблица со схемой, состоящей из колонок разных типов, описанных в `org.apache.spark.sql.types`  

### Обработка данных
**`RDD`**: сериализуемые функции  
**`DF`**: кодогенерация SQL > Java код  

### Функции и алгоритмы
**`RDD`**: нет ограничений  
**`DF`**: ограничен SQL операторами, функциями `org.apache.spark.sql.functions` и пользовательскими функциями  

### Источники данных 
**`RDD`**: каждый источник имеет свое API  
**`DF`**: единое API для всех источников 

### Производительность
**`RDD`**: напрямую зависит от качества кода   
**`DF`**: встроенные механизмы оптимизации SQL запроса   

### Потоковая обработка данных
**`RDD`**: устаревший DStreams   
**`DF`**: активно развивающийся Structured Streaming   

## Структура джобы Spark

+ Spark **«ничего не делает»** до тех пор, пока драйверная программа не вызовет действие
+ При каждом вызове действия планировщик Spark строит DAG выполнения и запускает задание Spark (Spark job)
+ Джоба Spark состоит из этапов, представляющих собой шаги преобразования данных, необходимые для формирования итогового набора RDD.
+ Этап состоит из набора tasks, каждая из которых означает параллельное вычисление, выполняемое на исполнителе.

## Dataframe API

**Dataframe:**
+ структурированная колоночная структура данных
+ может быть создана на основе:
  - локальной коллекции
  - файла (файлов)
  - базы данных
+ в python работает значительно быстрее, чем RDD
+ под капотом использует RDD
+ позволяет выполнять произвольные SQL операции с данными
+ аналогично RDD являются ленивыми и неизменяеыми

## Из чего состоит Dataframe
+ схема [pyspsark.sql.StructType](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.StructType)
+ колонки [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column)
+ данные [pyspark.sql.Row](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Row)

## Структура джобы Spark

+ Spark **«ничего не делает»** до тех пор, пока драйверная программа не вызовет действие
+ При каждом вызове действия планировщик Spark строит DAG выполнения и запускает задание Spark (Spark job)
+ Джоба Spark состоит из этапов, представляющих собой шаги преобразования данных, необходимые для формирования итогового набора RDD.
+ Этап состоит из набора tasks, каждая из которых означает параллельное вычисление, выполняемое на исполнителе.

In [5]:
# Данные те же, что и в дз
df = spark.read.csv('train.csv', header=True, inferSchema=True)

In [6]:
# посмотреть схему датафрейма (колонки и их типы)
# cхема может быть сложной, колонка может иметь массив и количество вложенности никак не лимитировано 
df.printSchema()

root
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)
 |-- price_range: integer (nullable = true)



In [7]:
# select - выбрать несколько столбцов для вывода в консоль драйвера
df.select("battery_power", "blue").show(10, False)

# Когда вы выбираете несколько столбцов из исходного датафрейма, вы не мутируете их, а создаете новый в памяти
# DF и RDD неизменяемые типы данных
df.show()

+-------------+----+
|battery_power|blue|
+-------------+----+
|842          |0   |
|1021         |1   |
|563          |1   |
|615          |1   |
|1821         |1   |
|1859         |0   |
|1821         |0   |
|1954         |0   |
|1445         |1   |
|509          |1   |
+-------------+----+
only showing top 10 rows

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|  

In [8]:
# write - запись в файл 

df.write.mode("overwrite").csv('my_data.csv')

## Чтение данных из источника
Основной метод чтения любых источников

```df = spark.read.format(datasource_type).option(datasource_options).load(object_name)```

+ ```datasource_type``` - тип источника ("parquet", "json", "cassandra") и т. д.
+ ```datasource_options``` - опции для работы с источником (логины, пароли, адреса для подключения и т. д.)
+ ```object_name``` - имя таблицы/файла/топика/индекса

[DataframeReader](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader):
+ по умолчанию выводит схему данных
+ является трансформацией (ленивый)
+ возвращает [Dataframe](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)

### Список (неполный) поддерживаемых источников данных
+ Файлы:
  - json
  - text
  - csv
  - orc
  - parquet
  - delta
+ Базы данных
  - elasticsearch
  - cassandra
  - jdbc
  - hive
  - redis
  - mongo
+ Брокеры сообщений
  - kafka
  

**Библиотеки для работы с источниками должны быть доступны в JAVA CLASSPATH на драйвере и воркерах!**

## Запись данных
Основной метод записи в любые системы

```df.write.format(datasource_type).options(datasource_options).mode(savemode).save(object_name)```

+ ```datasource_type``` - тип источника ("parquet", "json", "cassandra") и т. д.
+ ```datasource_options``` - опции для работы с источником (логины, пароли, адреса для подключения и т. д.)
+ ```savemode``` - режим записи данных (добавление, перезапись и т. д.)
+ ```object_name``` - имя таблицы/файла/топика/индекса

[DataFrameWriter](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter):
+ метод ```save``` является действием
+ позволяет работать с партиционированными данными (parquet, orc)
+ не всегда валидирует схему и формат данных


### Список (неполный) поддерживаемых источников данных
+ Файлы:
  - json
  - text
  - csv
  - orc
  - parquet
  - delta
+ Базы данных
  - elasticsearch
  - cassandra
  - jdbc
  - hive
  - redis
  - mongo
+ Брокеры сообщений
  - kafka
  

**Библиотеки для работы с источниками должны быть доступны в JAVA CLASSPATH на драйвере и воркерах!**




## Функции pyspark.sql.functions

Spark обладает достаточно большим набором встроенных функций, доступных в [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions), поэтому перед тем, как писать свою UDF, стоит обязательно поискать нужную функцию в данном пакете.

К тому же, все функции Spark принимают на вход и возвращают [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column), а это значит, что вы можете совмещать функции вместе

In [10]:
import pyspark.sql.functions as F

In [15]:
df = spark.range(0, 10)

my_awesome_col = F.struct(F.col("id")).alias("foo")

#cложная структура файла
df.select(my_awesome_col).printSchema()
df.select(my_awesome_col).show()

root
 |-- foo: struct (nullable = false)
 |    |-- id: long (nullable = false)

+---+
|foo|
+---+
|{0}|
|{1}|
|{2}|
|{3}|
|{4}|
|{5}|
|{6}|
|{7}|
|{8}|
|{9}|
+---+



## Работа с данными

In [16]:
# Удаляем дубликаты с помощью метода distinct(), который возвращает новый df, а не мутирует текущий
# Удаляет все колонки, где значения одинаковые во всех столбцах одинаковые
df.distinct().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [31]:
# есть более удобный вариант, где можно указать, по каким колонкам будет проводить удаление дубликатов
# уникальность будет определяться по кортежу из subset

#df.dropDuplicates(subset=["battery_power", "dual_sim"]).show()

df.show() # Нам показывает, что в тек. датафрейме остался один столбец ID.
#Поэтому перезапрашиваем df = spark.read.csv('train.csv', header=True, inferSchema=True), Либо JOIN 

# df.dropDuplicates(F.col["battery_power"]).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [32]:
df = spark.read.csv('train.csv', header=True, inferSchema=True)

In [33]:
df.dropDuplicates(subset=["battery_power", "dual_sim"]).show() # +

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|         1504|   0|        2.3|       1|  1|     0|        64|  0.9|      182|      1|  2|       19|    1923|3059|   6|   5|       10|      0|           1|   0|          3|
|          916|   1|        2.6|       0|  0|     0|        36|  0.3|      195|      4| 12|      188|     992|1470|  13|   0|       14|      0|           1|   0|          0|
|          914|   1|        0.9|       0|  2|     0|         4|  0.8|      100|      2| 12|      191|    1495| 808|  11|   7|     

In [35]:
df.dropDuplicates(subset=["battery_power"]).show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|         1645|   1|        1.8|       0|  1|     0|        21|  0.7|      123|      7|  2|      897|    1308| 783|  10|   8|       20|      1|           0|   0|          0|
|         1591|   1|        2.2|       0|  3|     1|        49|  0.4|      154|      6|  7|     1384|    1883|3703|  17|   6|       20|      1|           1|   0|          3|
|          833|   1|        2.2|       0|  4|     0|        27|  0.9|      124|      1|  6|      359|    1395|2150|   9|   5|     

In [36]:
# Удаляем пустые строки. Параметр "all" означает, что будут удалены только те строки, в которых ВСЕ элементы null
# Пакет na - разные методы на пропусками в данных

# .na.drop("all") - удаляем строки, где во всех столбцах значение Null
# если в na передать аргумент any, удалим строки, где встречаются хотя бы раз Null

df \
    .distinct() \
    .na.drop("all") \
    .show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          596|   0|        2.1|       1|  9|     0|        64|  0.8|      111|      8| 15|      885|    1854|3238|  16|  13|       10|      0|           0|   0|          3|
|         1414|   0|        2.0|       1|  2|     0|        54|  0.8|       99|      7|  3|      262|     829|3799|  17|   7|        7|      1|           0|   1|          3|
|          598|   0|        2.3|       1|  7|     0|        21|  0.7|      196|      2| 10|      680|    1173|3293|   8|   4|     

In [37]:
# Заполняем пустые значения, потому что 1 + 0 = 1, а 1 + NULL = NULL
# Передаем в na.fill python словарь, в котором указываются значения, которые следует использовать для null

df \
    .distinct() \
    .na.drop("all") \
    .na.fill({'dual_sim': 'Unknown', 'dual_sim': 0,}) \
    .show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          596|   0|        2.1|       1|  9|     0|        64|  0.8|      111|      8| 15|      885|    1854|3238|  16|  13|       10|      0|           0|   0|          3|
|         1414|   0|        2.0|       1|  2|     0|        54|  0.8|       99|      7|  3|      262|     829|3799|  17|   7|        7|      1|           0|   1|          3|
|          598|   0|        2.3|       1|  7|     0|        21|  0.7|      196|      2| 10|      680|    1173|3293|   8|   4|     